In [1]:
#WEBSITE ANALYSIS 
import time
import requests
import pandas as pd
from pandas import json_normalize
import json

#datas a serem requeridas:
start_date='2021-01' #YYYY-MM
end_date='2023-10'  #YYYY-MM

def process_dataframe(input_df, company, vertical, state,platform):
    # Create a new DataFrame with only the desired columns
    processed_df = input_df[['date', 'visits', 'bounce_rate', 'pages_per_visit',
                             'visit_duration', 'page_views', 'share', 'unique_visitors']].copy()
    
    # Add new columns with the provided inputs
    processed_df['company'] = company
    processed_df['vertical'] = vertical
    processed_df['state'] = state
    processed_df['platform'] = platform
    
    return processed_df

def transform_dataframe(original_df, company, vertical, state, platform):
    metric_name = original_df.columns[1]
    transformed_df = pd.DataFrame()
    transformed_df['date'] = original_df['date']
    transformed_df['metric_name'] = metric_name
    transformed_df['metric_value'] = original_df[metric_name]
    transformed_df['company'] = company
    transformed_df['vertical'] = vertical
    transformed_df['state'] = state
    transformed_df['platform'] = platform
    return transformed_df

def transform_dataframe_0(original_df, company, vertical, state, platform,metric_name):
    transformed_df = pd.DataFrame()
    transformed_df['date'] = original_df['date']
    transformed_df['metric_name'] = metric_name
    transformed_df['metric_value'] = original_df[original_df.columns[0]]
    transformed_df['company'] = company
    transformed_df['vertical'] = vertical
    transformed_df['state'] = state
    transformed_df['platform'] = platform
    return transformed_df

company_urls =[
'olx.com.br', 
'quintoandar.com.br',
'chavesnamao.com.br',
'icarros.com.br',
'imovelweb.com.br',
'kavak.com',
'loft.com.br',
'mobiauto.com.br',
'vivareal.com.br',
'webmotors.com.br',
'zapimoveis.com.br',
'casamineira.com.br', 
'meucarronovo.com.br',
'enjoei.com.br',
'elo7.com.br',
'mercadolivre.com.br',
'shopee.com.br'
]
company_names =[
    'olx',
'quinto_andar',
'chaves_na_mao',
'icarros',
'imovelweb',
'kavak',
'loft',
'mobiauto',
'vivareal',
'webmotors',
'zap',
'casa_mineira',
'meu_carro_novo',
'enjoei',
'elo7',
'mercado_livre',
'shopee'
]
vertical_names =[
'FULL',
'REAL_ESTATE',
'FULL',
'AUTOS',
'REAL_ESTATE',
'AUTOS',
'REAL_ESTATE',
'AUTOS',
'REAL_ESTATE',
'AUTOS',
'REAL_ESTATE',
'REAL_ESTATE',
'AUTOS',
'GOODS',
'GOODS',
'GOODS',
'GOODS'
]
company_map = dict(zip(company_urls, company_names)) #mapear uma lista na outra
vertical_map = dict(zip(company_urls, vertical_names)) #mapear uma lista na outra

# Replace with your actual API key
API_KEY = "c165c953a9fd44908addfcccec2c8caa"
# Headers with authorization
headers = {
    "Authorization": f"Bearer {API_KEY}"
}


# Colunas do DataFrame final
columns = ['date', 'metric_name', 'metric_value', 'company', 'vertical', 'state', 'platform']
# Criar um DataFrame vazio com as colunas especificadas
df_website_analysis = pd.DataFrame(columns=columns)
                                                ###############
sleep_time=0.5                                  ###Main LOOP###
for company_url in company_urls:                ###############
    company=company_map.get(company_url) 
    granularity='monthly'
    
    #################################################################################################################
                                                    #total

    vertical=vertical_map.get(company_url) 
    state='All'
    platform='Total'
    url_metric_names=[ 'bounce-rate', 'pages-per-visit','average-visit-duration', 'page-views']
    df_metric_names=[ 'bounce_rate', 'pages_per_visit','average_visit_duration', 'pages_views']
    metric_name_map = dict(zip(url_metric_names, df_metric_names)) #mapear uma lista na outra

    #visits
    url = 'https://api.similarweb.com/v1/website/'+company_url+'/total-traffic-and-engagement/visits?api_key=c165c953a9fd44908addfcccec2c8caa&start_date='+start_date+'&end_date='+end_date+'&country=br&granularity='+granularity+'&main_domain_only=false&format=json&show_verified=false&mtd=false&engaged_only=false'
    # Make the API request
    response = requests.get(url, headers={'api-key':'c165c953a9fd44908addfcccec2c8caa'})
    # Check if the request was successful
    if response.status_code == 200:
        data= response.json()
        time.sleep(sleep_time)
        # Process the data as needed
        print('Data for '+company+':'+platform+' TOTAL visits processed successfully!')
    else:
        print('API request failed for '+company+' :', response.status_code)

    metric_data = data["visits"]
    df = pd.DataFrame(metric_data)
    df["date"] = pd.to_datetime(df["date"])
    final_df=transform_dataframe(df, company, vertical, state, platform)
    
    #loop métricas
    for metric_name in url_metric_names:
        url = 'https://api.similarweb.com/v1/website/'+company_url+'/total-traffic-and-engagement/'+metric_name+'?api_key=c165c953a9fd44908addfcccec2c8caa&start_date='+start_date+'&end_date='+end_date+'&country=br&granularity='+granularity+'&main_domain_only=false&format=json&show_verified=false&mtd=false&engaged_only=false'
        # Make the API request
        response = requests.get(url, headers={'api-key':'c165c953a9fd44908addfcccec2c8caa'})
        # Check if the request was successful
        if response.status_code == 200:
            data= response.json()
            time.sleep(sleep_time)
            # Process the data as needed
            print('Data for '+company+':'+platform+'-'+metric_name+' processed successfully!')
        else:
            print('API request failed for '+company+' :', response.status_code)

        metric_data = data[metric_name_map.get(metric_name)]
        df = pd.DataFrame(metric_data)
        df["date"] = pd.to_datetime(df["date"])
        df=transform_dataframe(df, company, vertical, state, platform)
        final_df=pd.concat([final_df, df], ignore_index=True) # apenda novos dados

  
    total_df=final_df
   

    #################################################################################################################
                                                    #desktop

    vertical=vertical_map.get(company_url) 
    state='All'
    platform='Desktop'
    url_metric_names=[ 'bounce-rate', 'pages-per-visit','average-visit-duration', 'page-views']
    df_metric_names=[ 'bounce_rate', 'pages_per_visit','average_visit_duration', 'pages_views']
    metric_name_map = dict(zip(url_metric_names, df_metric_names)) #mapear uma lista na outra

    #visits
    url = 'https://api.similarweb.com/v1/website/'+company_url+'/traffic-and-engagement/visits?api_key=c165c953a9fd44908addfcccec2c8caa&start_date='+start_date+'&end_date='+end_date+'&country=br&granularity='+granularity+'&main_domain_only=false&format=json&mtd=false&engaged_only=false'

    # Make the API request
    response = requests.get(url, headers={'api-key':'c165c953a9fd44908addfcccec2c8caa'})
    # Check if the request was successful
    if response.status_code == 200:
        data= response.json()
        time.sleep(sleep_time)
        # Process the data as needed
        print('Data for '+company+':'+platform+' visits processed successfully!')
    else:
        print('API request failed for '+company+' :', response.status_code)

    metric_data = data["visits"]
    df = pd.DataFrame(metric_data)
    df["date"] = pd.to_datetime(df["date"])
    final_df=transform_dataframe(df, company, vertical, state, platform)
    
    #deduplicated audience
    url='https://api.similarweb.com/v1/website/'+company_url+'/dedup/deduplicated-audiences?api_key=c165c953a9fd44908addfcccec2c8caa&start_date='+start_date+'&end_date='+end_date+'&country=br&main_domain_only=false&format=json'
    # Make the API request
    response = requests.get(url, headers={'api-key':'c165c953a9fd44908addfcccec2c8caa'})
    # Check if the request was successful
    if response.status_code == 200:
        data= response.json()
        time.sleep(sleep_time)
        # Process the data as needed
        print('Data for '+company+':'+platform+' deduplicated audience processed successfully!')
    else:
        print('API request failed for '+company+' :', response.status_code)

    #esse bloco forma o dataframe dos dados de audiencia deduplicada
    # inicializando o dataframe vazio com as 4 colunas (a audiencia total e os 3 shares distintos)
    dataf = {
    'date': [],
    'total_deduplicated_audience': [],
    'desktop_only_share': [],
    'mobileweb_only_share': [],
    'desktop_and_mobileweb_share': []
    }
    #aqui preenchemos o dataframe
    for entry in data['data']:
        dataf['date'].append(entry['date'])
        dataf['total_deduplicated_audience'].append(entry['dedup_data']['total_deduplicated_audience'])
        dataf['desktop_only_share'].append(entry['dedup_data']['desktop_only_audience_share'])
        dataf['mobileweb_only_share'].append(entry['dedup_data']['mobile_web_only_audience_share'])
        dataf['desktop_and_mobileweb_share'].append(entry['dedup_data']['desktop_and_mobile_web_audience_share'])
    dedf = pd.DataFrame(dataf)

    #Aqui multiplicamos as colunas de share e audiencia pra formar o dataframe
    #desktop only
    metric_data= dedf["total_deduplicated_audience"]*dedf["desktop_only_share"]
    df = pd.DataFrame(metric_data)
    df["date"] = pd.to_datetime(dedf["date"])
    dedup_desktop_df=transform_dataframe_0(df, company, vertical, state, 'Desktop','deduplicated_audience')
    #mobileweb only
    metric_data= dedf["total_deduplicated_audience"]*dedf["mobileweb_only_share"]
    df = pd.DataFrame(metric_data)
    df["date"] = pd.to_datetime(dedf["date"])
    dedup_mobileweb_df=transform_dataframe_0(df, company, vertical, state, 'Mobile-web','deduplicated_audience')
    #mobileweb and desktop
    metric_data= dedf["total_deduplicated_audience"]*dedf["desktop_and_mobileweb_share"]
    df = pd.DataFrame(metric_data)
    df["date"] = pd.to_datetime(dedf["date"])
    dedup_total_df=transform_dataframe_0(df, company, vertical, state, 'Total','deduplicated_audience')
    final_df=pd.concat([final_df, dedup_total_df, dedup_mobileweb_df,dedup_desktop_df], ignore_index=True) # apenda novos dados

    
    #new and returning users
    url='https://api.similarweb.com/v1/website/'+company_url+'/audience/new-vs-returning?api_key=c165c953a9fd44908addfcccec2c8caa&start_date='+start_date+'&end_date='+end_date+'&country=br'
    # Make the API request
    response = requests.get(url, headers={'api-key':'c165c953a9fd44908addfcccec2c8caa'})
    # Check if the request was successful
    if response.status_code == 200:
        data= response.json()
        # Process the data as needed
        print('Data for '+company+':'+platform+' new and returning users desktop processed successfully!')
    else:
        print('API request failed for '+company+' :', response.status_code)
     #esse bloco forma o dataframe dos dados de audiencia deduplicada
        # inicializando o dataframe vazio com as 4 colunas (a audiencia total e os 3 shares distintos)
    data_list = [{'date': date, 'new_users': values['new_users']} for date, values in data['data'][company_url]['graph'].items()]
    df = pd.DataFrame(data_list)
    df["date"] = pd.to_datetime(df["date"])
    new_users_df=transform_dataframe(df, company, vertical, state, platform)
    #Returning users
    data_list = [{'date': date, 'returning_users': values['returning_users']} for date, values in data['data'][company_url]['graph'].items()]
    df = pd.DataFrame(data_list)
    df["date"] = pd.to_datetime(df["date"])
    returning_users_df=transform_dataframe(df, company, vertical, state, platform)
    final_df=pd.concat([final_df,new_users_df,returning_users_df], ignore_index=True) # apenda novos dados
    
    
    for metric_name in url_metric_names:
        url = 'https://api.similarweb.com/v1/website/'+company_url+'/traffic-and-engagement/'+metric_name+'?api_key=c165c953a9fd44908addfcccec2c8caa&start_date='+start_date+'&end_date='+end_date+'&country=br&granularity='+granularity+'&main_domain_only=false&format=json&mtd=false&engaged_only=false'

        # Make the API request
        response = requests.get(url, headers={'api-key':'c165c953a9fd44908addfcccec2c8caa'})
        # Check if the request was successful
        if response.status_code == 200:
            data= response.json()
            time.sleep(sleep_time)
            # Process the data as needed
            print('Data for '+company+':'+platform+'-'+metric_name+' processed successfully!')
        else:
            print('API request failed for '+company+' :', response.status_code)

        metric_data = data[metric_name_map.get(metric_name)]
        df = pd.DataFrame(metric_data)
        df["date"] = pd.to_datetime(df["date"])
        df=transform_dataframe(df, company, vertical, state, platform)
        final_df=pd.concat([final_df, df], ignore_index=True) # apenda novos dados

    #unique visitors
    url = 'https://api.similarweb.com/v1/website/'+company_url+'/unique-visitors/desktop_unique_visitors?api_key=c165c953a9fd44908addfcccec2c8caa&start_date='+start_date+'&end_date='+end_date+'&country=br&granularity='+granularity+'&main_domain_only=false&format=json&mtd=false&engaged_only=false'

    # Make the API request
    response = requests.get(url, headers={'api-key':'c165c953a9fd44908addfcccec2c8caa'})
    # Check if the request was successful
    if response.status_code == 200:
        data= response.json()
        time.sleep(sleep_time)
        # Process the data as needed
        print('Data for '+company+':'+platform+' unique-visitors processed successfully!')
    else:
        print('API request failed for '+company+' :', response.status_code)

    metric_data = data["unique_visitors"]
    df = pd.DataFrame(metric_data)
    df["date"] = pd.to_datetime(df["date"])
    df=transform_dataframe(df, company, vertical, state, platform)
    final_df=pd.concat([final_df, df], ignore_index=True) # apenda novos dados
    desktop_df=final_df
    #print(desktop_df)

    #################################################################################################################
                                                    #mobile-web

    vertical=vertical_map.get(company_url) 
    state='All'
    platform='Mobile-web'
    url_metric_names=[ 'bounce-rate', 'pages-per-visit','average-visit-duration', 'page-views']
    df_metric_names=[ 'bounce_rate', 'pages_per_visit','average_visit_duration', 'pages_views']
    metric_name_map = dict(zip(url_metric_names, df_metric_names)) #mapear uma lista na outra

    #visits
    url = 'https://api.similarweb.com/v2/website/'+company_url+'/mobile-web/visits?api_key=c165c953a9fd44908addfcccec2c8caa&start_date='+start_date+'&end_date='+end_date+'&country=br&granularity='+granularity+'&main_domain_only=false&format=json&mtd=false&engaged_only=false'

    # Make the API request
    response = requests.get(url, headers={'api-key':'c165c953a9fd44908addfcccec2c8caa'})
    # Check if the request was successful
    if response.status_code == 200:
        data= response.json()
        time.sleep(sleep_time)
        # Process the data as needed
        print('Data for '+company+':'+platform+' visits processed successfully!')
    else:
        print('API request failed for '+company+' :', response.status_code)

    metric_data = data["visits"]
    df = pd.DataFrame(metric_data)
    df["date"] = pd.to_datetime(df["date"])
    final_df=transform_dataframe(df, company, vertical, state, platform)
    

    for metric_name in url_metric_names:
        url = 'https://api.similarweb.com/v2/website/'+company_url+'/mobile-web/'+metric_name+'?api_key=c165c953a9fd44908addfcccec2c8caa&start_date='+start_date+'&end_date='+end_date+'&country=br&granularity='+granularity+'&main_domain_only=false&format=json&mtd=false&engaged_only=false'

        # Make the API request
        response = requests.get(url, headers={'api-key':'c165c953a9fd44908addfcccec2c8caa'})
        # Check if the request was successful
        if response.status_code == 200:
            data= response.json()
            time.sleep(sleep_time)
            # Process the data as needed
            print('Data for '+company+':'+platform+'-'+metric_name+' processed successfully!')
        else:
            print('API request failed for '+company+' :', response.status_code)

        metric_data = data[metric_name_map.get(metric_name)]
        df = pd.DataFrame(metric_data)
        df["date"] = pd.to_datetime(df["date"])
        df=transform_dataframe(df, company, vertical, state, platform)
        final_df=pd.concat([final_df, df], ignore_index=True) # apenda novos dados

    #unique visitors
    url = 'https://api.similarweb.com/v1/website/'+company_url+'/unique-visitors/mobileweb_unique_visitors?api_key=c165c953a9fd44908addfcccec2c8caa&start_date='+start_date+'&end_date='+end_date+'&country=br&granularity='+granularity+'&main_domain_only=false&format=json&mtd=false&engaged_only=false'

    # Make the API request
    response = requests.get(url, headers={'api-key':'c165c953a9fd44908addfcccec2c8caa'})
    # Check if the request was successful
    if response.status_code == 200:
        data= response.json()
        time.sleep(sleep_time)
        # Process the data as needed
        print('Data for '+company+':'+platform+' unique-visitors processed successfully!')
    else:
        print('API request failed for '+company+' :', response.status_code)

    metric_data = data["unique_visitors"]
    df = pd.DataFrame(metric_data)
    ##### NICOLLE: Crie uma coluna "date", a partir das colunas de ano e mes, no formato "YYYY-MM-DD"
    ##### E depois Rode essa linha abaixo pra converter a coluna date para o formato datetime
    df["date"] = pd.to_datetime(df["date"])
    df=transform_dataframe(df, company, vertical, state, platform)
    final_df=pd.concat([final_df, df], ignore_index=True) # apenda novos dados
    mweb_df=final_df
    #print(mweb_df)
    df_website_analysis=pd.concat([df_website_analysis, mweb_df,desktop_df,total_df], ignore_index=True) # apenda novos dados

df_website_analysis.to_csv("dataframe_website_analysis.csv", index=False)
df=df_website_analysis
for col in df.columns:
    if df[col].dtype == 'object':
        df = df.astype({col:'string'})
for col in df.columns:
    if df[col].dtype == 'float64':
        df = df.astype({col:'float32'})
df.rename(columns={'date': 'dt'}, inplace=True)
df.rename(columns={'platform': 'device'}, inplace=True)
df['dt'] = pd.to_datetime(df['dt'])
# Create 'year' and 'month' columns
df['year'] = df['dt'].dt.year.astype('int32')
df['month'] = df['dt'].dt.month.astype('int32')
mapeamento = {
    'olx': 'OLX',
    'quintoandar': 'QUINTOANDAR',
    'chaves_na_mao': 'CHAVESNAMAO',
    'icarros': 'ICARROS',
    'imovelweb': 'IMOVELWEB',
    'kavak': 'KAVAK',
    'loft': 'LOFT',
    'mobiauto': 'MOBIAUTO',
    'vivareal': 'VIVAREAL',
    'webmotors': 'WEBMOTORS',
    'zap': 'ZAPIMOVEIS'
}

# Substitua os valores na coluna 'company'
df['company'] = df['company'].replace(mapeamento)
mapeamento_2 = {
    'average_visit_duration': 'visit_duration',
    'pages_views': 'page_views'
}

# Substitua os valores na coluna 'metric_name'
df['metric_name'] = df['metric_name'].replace(mapeamento_2)
df_website_analysis=df

Data for olx:Total TOTAL visits processed successfully!
Data for olx:Total-bounce-rate processed successfully!
Data for olx:Total-pages-per-visit processed successfully!
Data for olx:Total-average-visit-duration processed successfully!
Data for olx:Total-page-views processed successfully!
Data for olx:Desktop visits processed successfully!
Data for olx:Desktop deduplicated audience processed successfully!
Data for olx:Desktop new and returning users desktop processed successfully!
Data for olx:Desktop-bounce-rate processed successfully!
Data for olx:Desktop-pages-per-visit processed successfully!
Data for olx:Desktop-average-visit-duration processed successfully!
Data for olx:Desktop-page-views processed successfully!
Data for olx:Desktop unique-visitors processed successfully!
Data for olx:Mobile-web visits processed successfully!
Data for olx:Mobile-web-bounce-rate processed successfully!
Data for olx:Mobile-web-pages-per-visit processed successfully!
Data for olx:Mobile-web-average-

In [2]:
#SEGMENT ANALYSIS - Segmentos antigos
import requests
import pandas as pd
from pandas import json_normalize
import json

def process_dataframe(input_df, company, vertical, state,platform):
    # Create a new DataFrame with only the desired columns
    processed_df = input_df[['date', 'visits', 'bounce_rate', 'pages_per_visit',
                             'visit_duration', 'page_views', 'share', 'unique_visitors']].copy()
    
    # Add new columns with the provided inputs
    processed_df['company'] = company
    processed_df['vertical'] = vertical
    processed_df['state'] = state
    processed_df['platform'] = platform
    
    return processed_df

def s_transform_dataframe(original_df, company, vertical, state, platform):
    metric_name = original_df.columns[0]
    transformed_df = pd.DataFrame()
    transformed_df['date'] = original_df['date']
    transformed_df['metric_name'] = metric_name
    transformed_df['metric_value'] = original_df[metric_name]
    transformed_df['company'] = company
    transformed_df['vertical'] = vertical
    transformed_df['state'] = state
    transformed_df['platform'] = platform
    return transformed_df


# Replace with your actual API key
API_KEY = "c165c953a9fd44908addfcccec2c8caa"
# Headers with authorization
headers = {
    "Authorization": f"Bearer {API_KEY}"
}

#start_date='2021-01' #YYYY-MM
#end_date='2023-07'  #YYYY-MM
#definir um sleeptime pra não onerar o request limit
sleep_time_s=0.5
                                                ###############
                                                ###BLOCO OLX###
                                                ###############
company='olx' 
company_url='olx.com.br'
granularity='monthly'
#################################################################################################################
                                                #segment_analysis
state='All'
platform='Desktop'
s_names=[ 'Goods', 
         'jobs&services',
         'Imóveis', 
         'Autos', 
         'Imóveis Aluguel', 
         'Imóveis Venda'
        ]
s_ids=[ '0064132f-bd03-4aa0-8004-658dadd6a895', 
       '0952e12d-6978-4b1d-9549-4d8742d159c9',
       '42a3a1c7-61bb-43b1-a2b1-71337b868251', 
       '5b8ee014-6798-41d7-a826-14267fabf7ec',
       '72ccd4a7-951a-46d3-845d-02e99ad81842',
       'e6c0f1b6-6a4b-433d-99c3-005371fcdba3'
      ]
s_metric_names=['visits',
              'bounce_rate',
              'pages_per_visit',
              'visit_duration',
              'page_views',
              'share',
              'unique_visitors'
              ]
vertical_map = dict(zip(s_ids, s_names)) #mapear uma lista na outra
#iter0
vertical=vertical_map.get(s_ids[0])
url= 'https://api.similarweb.com/v1/segment/'+s_ids[0]+'/traffic-and-engagement/query?api_key=c165c953a9fd44908addfcccec2c8caa&start_date='+start_date+'&end_date='+end_date+'&country=br&metrics=visits%2Cshare%2Cpages-per-visit%2Cpage-views%2Cbounce-rate%2Cvisit-duration%2Cunique-visitors&granularity=monthly&format=json&mtd=false'

# Make the API request
response = requests.get(url, headers={'api-key':'c165c953a9fd44908addfcccec2c8caa'})
# Check if the request was successful
if response.status_code == 200:
    json_data= response.json()
    data = json_data["segments"]
    df = pd.DataFrame(data)
    # Process the data as needed
    print('Data for '+vertical_map.get(s_ids[0])+' processed successfully!')
else:
    print('API request failed for '+vertical_map.get(s_ids[0])+' :', response.status_code)

#Iter0.0
s_metric_names=['visits',
              'bounce_rate',
              'pages_per_visit',
              'visit_duration',
              'page_views',
              'share',
              'unique_visitors'
              ]
metric_data = df[s_metric_names[0]]
mdf = pd.DataFrame(metric_data)
mdf["date"] = pd.to_datetime(df["date"])
final_df=s_transform_dataframe(mdf, company, vertical, state, platform) 
s_metric_names.pop(0)
#loop0.0
for metrics in s_metric_names:
    metric_data = df[metrics]
    mdf = pd.DataFrame(metric_data)
    mdf["date"] = pd.to_datetime(df["date"])
    mdf=s_transform_dataframe(mdf, company, vertical, state, platform)
    final_df=pd.concat([final_df, mdf], ignore_index=True) # apenda novos dados



#loop
s_ids.pop(0)
for ids in s_ids:
    vertical=vertical_map.get(ids)
    url= 'https://api.similarweb.com/v1/segment/'+ids+'/traffic-and-engagement/query?api_key=c165c953a9fd44908addfcccec2c8caa&start_date='+start_date+'&end_date='+end_date+'&country=br&metrics=visits%2Cshare%2Cpages-per-visit%2Cpage-views%2Cbounce-rate%2Cvisit-duration%2Cunique-visitors&granularity=monthly&format=json&mtd=false'

     # Make the API request
    response = requests.get(url, headers={'api-key':'c165c953a9fd44908addfcccec2c8caa'})
    # Check if the request was successful
    if response.status_code == 200:
        json_data= response.json()
        data = json_data["segments"]
        df = pd.DataFrame(data)
        # Process the data as needed
        print('Data for '+vertical_map.get(ids)+' processed successfully!')
    else:
        print('API request failed for '+vertical_map.get(ids)+' :', response.status_code)
        
    #Iter0.0 
    s_metric_names=['visits',
              'bounce_rate',
              'pages_per_visit',
              'visit_duration',
              'page_views',
              'share',
              'unique_visitors'
              ]
    metric_data = df[s_metric_names[0]]
    mdf = pd.DataFrame(metric_data)
    mdf["date"] = pd.to_datetime(df["date"])
    semi_final_df=s_transform_dataframe(mdf, company, vertical, state, platform) 
    s_metric_names.pop(0)
    #loop0.0
    for metrics in s_metric_names:
        metric_data = df[metrics]
        mdf = pd.DataFrame(metric_data)
        mdf["date"] = pd.to_datetime(df["date"])
        mdf=s_transform_dataframe(mdf, company, vertical, state, platform)
        semi_final_df=pd.concat([semi_final_df, mdf], ignore_index=True) # apenda novos dados
    final_df=pd.concat([final_df, semi_final_df], ignore_index=True) # apenda novos dados

olx_segments_df=final_df
time.sleep(sleep_time_s)
#print(olx_segments_df)

                                                ###############
                                             ###BLOCO IMOVELWEB###
                                                ###############
company='imovelweb' 
company_url='imovelweb.com.br'
granularity='monthly'
#################################################################################################################
                                                #segment_analysis
state='All'
platform='Desktop'
s_names=[ 
'Leilão',
'Lançamentos',
'Temporada',
'Comprar',
'Aluguel'
        ]
s_ids=[
'4381ae5e-7a3c-4cfa-bcf1-6e05dcd19c16',
'4fd76736-e12e-47fb-a0c6-26df24f153d5',
'53f79467-177a-4006-b620-20c0ca919236',
'acfda8e7-b32e-4fd8-ada1-40fe06ef2591',
'f3d1a778-fd81-4c1a-8e5a-3c8015f45a04'
      ]
s_metric_names=['visits',
              'bounce_rate',
              'pages_per_visit',
              'visit_duration',
              'page_views',
              'share',
              'unique_visitors'
              ]
vertical_map = dict(zip(s_ids, s_names)) #mapear uma lista na outra
#iter0
vertical=vertical_map.get(s_ids[0])
url= 'https://api.similarweb.com/v1/segment/'+s_ids[0]+'/traffic-and-engagement/query?api_key=c165c953a9fd44908addfcccec2c8caa&start_date='+start_date+'&end_date='+end_date+'&country=br&metrics=visits%2Cshare%2Cpages-per-visit%2Cpage-views%2Cbounce-rate%2Cvisit-duration%2Cunique-visitors&granularity=monthly&format=json&mtd=false'

# Make the API request
response = requests.get(url, headers={'api-key':'c165c953a9fd44908addfcccec2c8caa'})
# Check if the request was successful
if response.status_code == 200:
    json_data= response.json()
    data = json_data["segments"]
    df = pd.DataFrame(data)
    # Process the data as needed
    print('Data for '+vertical_map.get(s_ids[0])+' processed successfully!')
else:
    print('API request failed for '+vertical_map.get(s_ids[0])+' :', response.status_code)

#Iter0.0
s_metric_names=['visits',
              'bounce_rate',
              'pages_per_visit',
              'visit_duration',
              'page_views',
              'share',
              'unique_visitors'
              ]
metric_data = df[s_metric_names[0]]
mdf = pd.DataFrame(metric_data)
mdf["date"] = pd.to_datetime(df["date"])
final_df=s_transform_dataframe(mdf, company, vertical, state, platform) 
s_metric_names.pop(0)
#loop0.0
for metrics in s_metric_names:
    metric_data = df[metrics]
    mdf = pd.DataFrame(metric_data)
    mdf["date"] = pd.to_datetime(df["date"])
    mdf=s_transform_dataframe(mdf, company, vertical, state, platform)
    final_df=pd.concat([final_df, mdf], ignore_index=True) # apenda novos dados



#loop
s_ids.pop(0)
for ids in s_ids:
    vertical=vertical_map.get(ids)
    url= 'https://api.similarweb.com/v1/segment/'+ids+'/traffic-and-engagement/query?api_key=c165c953a9fd44908addfcccec2c8caa&start_date='+start_date+'&end_date='+end_date+'&country=br&metrics=visits%2Cshare%2Cpages-per-visit%2Cpage-views%2Cbounce-rate%2Cvisit-duration%2Cunique-visitors&granularity=monthly&format=json&mtd=false'

     # Make the API request
    response = requests.get(url, headers={'api-key':'c165c953a9fd44908addfcccec2c8caa'})
    # Check if the request was successful
    if response.status_code == 200:
        json_data= response.json()
        data = json_data["segments"]
        df = pd.DataFrame(data)
        # Process the data as needed
        print('Data for '+vertical_map.get(ids)+' processed successfully!')
    else:
        print('API request failed for '+vertical_map.get(ids)+' :', response.status_code)
        
    #Iter0.0 
    s_metric_names=['visits',
              'bounce_rate',
              'pages_per_visit',
              'visit_duration',
              'page_views',
              'share',
              'unique_visitors'
              ]
    metric_data = df[s_metric_names[0]]
    mdf = pd.DataFrame(metric_data)
    mdf["date"] = pd.to_datetime(df["date"])
    semi_final_df=s_transform_dataframe(mdf, company, vertical, state, platform) 
    s_metric_names.pop(0)
    #loop0.0
    for metrics in s_metric_names:
        metric_data = df[metrics]
        mdf = pd.DataFrame(metric_data)
        mdf["date"] = pd.to_datetime(df["date"])
        mdf=s_transform_dataframe(mdf, company, vertical, state, platform)
        semi_final_df=pd.concat([semi_final_df, mdf], ignore_index=True) # apenda novos dados
    final_df=pd.concat([final_df, semi_final_df], ignore_index=True) # apenda novos dados

imovelweb_segments_df=final_df
time.sleep(sleep_time_s)
#print(imovelweb_segments_df)

                                                ###############
                                             ###BLOCO QUINTO ANDAR###
                                                ###############
company='quintoandar' 
company_url='quintoandar.com.br'
granularity='monthly'
#################################################################################################################
                                                #segment_analysis
state='All'
platform='Desktop'
s_names=[ 
'Alugar',
'Comprar'
        ]
s_ids=[
'487abf11-54a8-436b-9a1a-89f5d116ef4f',
'5037d554-13e0-4e68-86fa-513ee8799e5d'
      ]
s_metric_names=['visits',
              'bounce_rate',
              'pages_per_visit',
              'visit_duration',
              'page_views',
              'share',
              'unique_visitors'
              ]
vertical_map = dict(zip(s_ids, s_names)) #mapear uma lista na outra
#iter0
vertical=vertical_map.get(s_ids[0])
url= 'https://api.similarweb.com/v1/segment/'+s_ids[0]+'/traffic-and-engagement/query?api_key=c165c953a9fd44908addfcccec2c8caa&start_date='+start_date+'&end_date='+end_date+'&country=br&metrics=visits%2Cshare%2Cpages-per-visit%2Cpage-views%2Cbounce-rate%2Cvisit-duration%2Cunique-visitors&granularity=monthly&format=json&mtd=false'

# Make the API request
response = requests.get(url, headers={'api-key':'c165c953a9fd44908addfcccec2c8caa'})
# Check if the request was successful
if response.status_code == 200:
    json_data= response.json()
    data = json_data["segments"]
    df = pd.DataFrame(data)
    # Process the data as needed
    print('Data for '+vertical_map.get(s_ids[0])+' processed successfully!')
else:
    print('API request failed for '+vertical_map.get(s_ids[0])+' :', response.status_code)

#Iter0.0
s_metric_names=['visits',
              'bounce_rate',
              'pages_per_visit',
              'visit_duration',
              'page_views',
              'share',
              'unique_visitors'
              ]
metric_data = df[s_metric_names[0]]
mdf = pd.DataFrame(metric_data)
mdf["date"] = pd.to_datetime(df["date"])
final_df=s_transform_dataframe(mdf, company, vertical, state, platform) 
s_metric_names.pop(0)
#loop0.0
for metrics in s_metric_names:
    metric_data = df[metrics]
    mdf = pd.DataFrame(metric_data)
    mdf["date"] = pd.to_datetime(df["date"])
    mdf=s_transform_dataframe(mdf, company, vertical, state, platform)
    final_df=pd.concat([final_df, mdf], ignore_index=True) # apenda novos dados



#loop
s_ids.pop(0)
for ids in s_ids:
    vertical=vertical_map.get(ids)
    url= 'https://api.similarweb.com/v1/segment/'+ids+'/traffic-and-engagement/query?api_key=c165c953a9fd44908addfcccec2c8caa&start_date='+start_date+'&end_date='+end_date+'&country=br&metrics=visits%2Cshare%2Cpages-per-visit%2Cpage-views%2Cbounce-rate%2Cvisit-duration%2Cunique-visitors&granularity=monthly&format=json&mtd=false'

     # Make the API request
    response = requests.get(url, headers={'api-key':'c165c953a9fd44908addfcccec2c8caa'})
    # Check if the request was successful
    if response.status_code == 200:
        json_data= response.json()
        data = json_data["segments"]
        df = pd.DataFrame(data)
        # Process the data as needed
        print('Data for '+vertical_map.get(ids)+' processed successfully!')
    else:
        print('API request failed for '+vertical_map.get(ids)+' :', response.status_code)
        
    #Iter0.0 
    s_metric_names=['visits',
              'bounce_rate',
              'pages_per_visit',
              'visit_duration',
              'page_views',
              'share',
              'unique_visitors'
              ]
    metric_data = df[s_metric_names[0]]
    mdf = pd.DataFrame(metric_data)
    mdf["date"] = pd.to_datetime(df["date"])
    semi_final_df=s_transform_dataframe(mdf, company, vertical, state, platform) 
    s_metric_names.pop(0)
    #loop0.0
    for metrics in s_metric_names:
        metric_data = df[metrics]
        mdf = pd.DataFrame(metric_data)
        mdf["date"] = pd.to_datetime(df["date"])
        mdf=s_transform_dataframe(mdf, company, vertical, state, platform)
        semi_final_df=pd.concat([semi_final_df, mdf], ignore_index=True) # apenda novos dados
    final_df=pd.concat([final_df, semi_final_df], ignore_index=True) # apenda novos dados

quintoandar_segments_df=final_df
time.sleep(sleep_time_s)
#print(quintoandar_segments_df)

                                                ###############
                                                ###BLOCO ZAP###
                                                ###############
company='zap' 
company_url='zapimoveis.com.br'
granularity='monthly'
#################################################################################################################
                                                #segment_analysis
state='All'
platform='Desktop'
s_names=[ 
'Lançamentos',
'Venda',
'Aluguel'
        ]
s_ids=[
'137b8b1f-5540-486c-9732-b8140fea6016',
'99787bac-64f3-4a16-8da3-ffd09e6b47fd',
'c17baaa4-72ce-44c4-9d6e-b9bd58a07592'
      ]
s_metric_names=['visits',
              'bounce_rate',
              'pages_per_visit',
              'visit_duration',
              'page_views',
              'share',
              'unique_visitors'
              ]
vertical_map = dict(zip(s_ids, s_names)) #mapear uma lista na outra
#iter0
vertical=vertical_map.get(s_ids[0])
url= 'https://api.similarweb.com/v1/segment/'+s_ids[0]+'/traffic-and-engagement/query?api_key=c165c953a9fd44908addfcccec2c8caa&start_date='+start_date+'&end_date='+end_date+'&country=br&metrics=visits%2Cshare%2Cpages-per-visit%2Cpage-views%2Cbounce-rate%2Cvisit-duration%2Cunique-visitors&granularity=monthly&format=json&mtd=false'

# Make the API request
response = requests.get(url, headers={'api-key':'c165c953a9fd44908addfcccec2c8caa'})
# Check if the request was successful
if response.status_code == 200:
    json_data= response.json()
    data = json_data["segments"]
    df = pd.DataFrame(data)
    # Process the data as needed
    print('Data for '+vertical_map.get(s_ids[0])+' processed successfully!')
else:
    print('API request failed for '+vertical_map.get(s_ids[0])+' :', response.status_code)

#Iter0.0
s_metric_names=['visits',
              'bounce_rate',
              'pages_per_visit',
              'visit_duration',
              'page_views',
              'share',
              'unique_visitors'
              ]
metric_data = df[s_metric_names[0]]
mdf = pd.DataFrame(metric_data)
mdf["date"] = pd.to_datetime(df["date"])
final_df=s_transform_dataframe(mdf, company, vertical, state, platform) 
s_metric_names.pop(0)
#loop0.0
for metrics in s_metric_names:
    metric_data = df[metrics]
    mdf = pd.DataFrame(metric_data)
    mdf["date"] = pd.to_datetime(df["date"])
    mdf=s_transform_dataframe(mdf, company, vertical, state, platform)
    final_df=pd.concat([final_df, mdf], ignore_index=True) # apenda novos dados



#loop
s_ids.pop(0)
for ids in s_ids:
    vertical=vertical_map.get(ids)
    url= 'https://api.similarweb.com/v1/segment/'+ids+'/traffic-and-engagement/query?api_key=c165c953a9fd44908addfcccec2c8caa&start_date='+start_date+'&end_date='+end_date+'&country=br&metrics=visits%2Cshare%2Cpages-per-visit%2Cpage-views%2Cbounce-rate%2Cvisit-duration%2Cunique-visitors&granularity=monthly&format=json&mtd=false'

     # Make the API request
    response = requests.get(url, headers={'api-key':'c165c953a9fd44908addfcccec2c8caa'})
    # Check if the request was successful
    if response.status_code == 200:
        json_data= response.json()
        data = json_data["segments"]
        df = pd.DataFrame(data)
        # Process the data as needed
        print('Data for '+vertical_map.get(ids)+' processed successfully!')
    else:
        print('API request failed for '+vertical_map.get(ids)+' :', response.status_code)
        
    #Iter0.0 
    s_metric_names=['visits',
              'bounce_rate',
              'pages_per_visit',
              'visit_duration',
              'page_views',
              'share',
              'unique_visitors'
              ]
    metric_data = df[s_metric_names[0]]
    mdf = pd.DataFrame(metric_data)
    mdf["date"] = pd.to_datetime(df["date"])
    semi_final_df=s_transform_dataframe(mdf, company, vertical, state, platform) 
    s_metric_names.pop(0)
    #loop0.0
    for metrics in s_metric_names:
        metric_data = df[metrics]
        mdf = pd.DataFrame(metric_data)
        mdf["date"] = pd.to_datetime(df["date"])
        mdf=s_transform_dataframe(mdf, company, vertical, state, platform)
        semi_final_df=pd.concat([semi_final_df, mdf], ignore_index=True) # apenda novos dados
    final_df=pd.concat([final_df, semi_final_df], ignore_index=True) # apenda novos dados

zap_segments_df=final_df
time.sleep(sleep_time_s)
#print(zap_segments_df)

                                                ###############
                                              ###BLOCO VIVAREAL###
                                                ###############
company='vivareal' 
company_url='vivareal.com.br'
granularity='monthly'
#################################################################################################################
                                                #segment_analysis
state='All'
platform='Desktop'
s_names=[ 
'Aluguel',
'Venda',
'Lançamentos'
        ]
s_ids=[
'4c3fa6c7-74db-417b-9939-84289e0b2aef',
'c003c9f9-de24-4bfb-b1cf-98e5fea945e0',
'ec4f1544-7ef0-40c7-b055-105fbad4e8e3'
      ]
s_metric_names=['visits',
              'bounce_rate',
              'pages_per_visit',
              'visit_duration',
              'page_views',
              'share',
              'unique_visitors'
              ]
vertical_map = dict(zip(s_ids, s_names)) #mapear uma lista na outra
#iter0
vertical=vertical_map.get(s_ids[0])
url= 'https://api.similarweb.com/v1/segment/'+s_ids[0]+'/traffic-and-engagement/query?api_key=c165c953a9fd44908addfcccec2c8caa&start_date='+start_date+'&end_date='+end_date+'&country=br&metrics=visits%2Cshare%2Cpages-per-visit%2Cpage-views%2Cbounce-rate%2Cvisit-duration%2Cunique-visitors&granularity=monthly&format=json&mtd=false'

# Make the API request
response = requests.get(url, headers={'api-key':'c165c953a9fd44908addfcccec2c8caa'})
# Check if the request was successful
if response.status_code == 200:
    json_data= response.json()
    data = json_data["segments"]
    df = pd.DataFrame(data)
    # Process the data as needed
    print('Data for '+vertical_map.get(s_ids[0])+' processed successfully!')
else:
    print('API request failed for '+vertical_map.get(s_ids[0])+' :', response.status_code)

#Iter0.0
s_metric_names=['visits',
              'bounce_rate',
              'pages_per_visit',
              'visit_duration',
              'page_views',
              'share',
              'unique_visitors'
              ]
metric_data = df[s_metric_names[0]]
mdf = pd.DataFrame(metric_data)
mdf["date"] = pd.to_datetime(df["date"])
final_df=s_transform_dataframe(mdf, company, vertical, state, platform) 
s_metric_names.pop(0)
#loop0.0
for metrics in s_metric_names:
    metric_data = df[metrics]
    mdf = pd.DataFrame(metric_data)
    mdf["date"] = pd.to_datetime(df["date"])
    mdf=s_transform_dataframe(mdf, company, vertical, state, platform)
    final_df=pd.concat([final_df, mdf], ignore_index=True) # apenda novos dados



#loop
s_ids.pop(0)
for ids in s_ids:
    vertical=vertical_map.get(ids)
    url= 'https://api.similarweb.com/v1/segment/'+ids+'/traffic-and-engagement/query?api_key=c165c953a9fd44908addfcccec2c8caa&start_date='+start_date+'&end_date='+end_date+'&country=br&metrics=visits%2Cshare%2Cpages-per-visit%2Cpage-views%2Cbounce-rate%2Cvisit-duration%2Cunique-visitors&granularity=monthly&format=json&mtd=false'

     # Make the API request
    response = requests.get(url, headers={'api-key':'c165c953a9fd44908addfcccec2c8caa'})
    # Check if the request was successful
    if response.status_code == 200:
        json_data= response.json()
        data = json_data["segments"]
        df = pd.DataFrame(data)
        # Process the data as needed
        print('Data for '+vertical_map.get(ids)+' processed successfully!')
    else:
        print('API request failed for '+vertical_map.get(ids)+' :', response.status_code)
        
    #Iter0.0 
    s_metric_names=['visits',
              'bounce_rate',
              'pages_per_visit',
              'visit_duration',
              'page_views',
              'share',
              'unique_visitors'
              ]
    metric_data = df[s_metric_names[0]]
    mdf = pd.DataFrame(metric_data)
    mdf["date"] = pd.to_datetime(df["date"])
    semi_final_df=s_transform_dataframe(mdf, company, vertical, state, platform) 
    s_metric_names.pop(0)
    #loop0.0
    for metrics in s_metric_names:
        metric_data = df[metrics]
        mdf = pd.DataFrame(metric_data)
        mdf["date"] = pd.to_datetime(df["date"])
        mdf=s_transform_dataframe(mdf, company, vertical, state, platform)
        semi_final_df=pd.concat([semi_final_df, mdf], ignore_index=True) # apenda novos dados
    final_df=pd.concat([final_df, semi_final_df], ignore_index=True) # apenda novos dados

vivareal_segments_df=final_df
time.sleep(sleep_time_s)
#print(vivareal_segments_df)


df=pd.concat([olx_segments_df, quintoandar_segments_df,imovelweb_segments_df,zap_segments_df,vivareal_segments_df], ignore_index=True) # apenda novos dados
for col in df.columns:
    if df[col].dtype == 'object':
        df = df.astype({col:'string'})
for col in df.columns:
    if df[col].dtype == 'float64':
        df = df.astype({col:'float32'})
df.rename(columns={'date': 'dt'}, inplace=True)
df['dt'] = pd.to_datetime(df['dt'])
# Create 'year' and 'month' columns
df['year'] = df['dt'].dt.year.astype('int32')
df['month'] = df['dt'].dt.month.astype('int32')
df.rename(columns={'platform': 'device'}, inplace=True)
df_segment_analysis=df

print('segment analysis finished processing sucessfully!')


Data for Goods processed successfully!
Data for jobs&services processed successfully!
Data for Imóveis processed successfully!
Data for Autos processed successfully!
Data for Imóveis Aluguel processed successfully!
Data for Imóveis Venda processed successfully!
Data for Leilão processed successfully!
Data for Lançamentos processed successfully!
Data for Temporada processed successfully!
Data for Comprar processed successfully!
Data for Aluguel processed successfully!
Data for Alugar processed successfully!
Data for Comprar processed successfully!
Data for Lançamentos processed successfully!
Data for Venda processed successfully!
Data for Aluguel processed successfully!
Data for Aluguel processed successfully!
Data for Venda processed successfully!
Data for Lançamentos processed successfully!
segment analysis finished processing sucessfully!


In [3]:
#SEGMENT ANALYSIS - Segmentos por estado
import requests
import pandas as pd
from pandas import json_normalize
import json

def process_dataframe(input_df, company, vertical, state,platform):
    # Create a new DataFrame with only the desired columns
    processed_df = input_df[['date', 'visits', 'bounce_rate', 'pages_per_visit',
                             'visit_duration', 'page_views', 'share', 'unique_visitors']].copy()
    
    # Add new columns with the provided inputs
    processed_df['company'] = company
    processed_df['vertical'] = vertical
    processed_df['state'] = state
    processed_df['platform'] = platform
    
    return processed_df

def s_transform_dataframe(original_df, company, vertical, state, platform):
    metric_name = original_df.columns[0]
    transformed_df = pd.DataFrame()
    transformed_df['date'] = original_df['date']
    transformed_df['metric_name'] = metric_name
    transformed_df['metric_value'] = original_df[metric_name]
    transformed_df['company'] = company
    transformed_df['vertical'] = vertical
    transformed_df['state'] = state
    transformed_df['device'] = device
    return transformed_df


# Replace with your actual API key
API_KEY = "c165c953a9fd44908addfcccec2c8caa"
# Headers with authorization
headers = {
    "Authorization": f"Bearer {API_KEY}"
}

#start_date='2021-01' #YYYY-MM
#end_date='2023-07'  #YYYY-MM
#definir um sleeptime pra não onerar o request limit
sleep_time_s=0.5

#dataframe contendo os segmentos definidos 
list_segments_df = pd.read_csv('segmentos_sweb_2023-11-10.csv')
#lista de nomes de métricas
s_metric_names=['visits',
                  'bounce_rate',
                  'pages_per_visit',
                  'visit_duration',
                  'page_views',
                  'share',
                  'unique_visitors'
                  ]
#Crie um dataframe vazio com as colunas de interesse
final_df = pd.DataFrame(columns=['company', 
                           'vertical', 
                           'metric_name', 
                           'metric_value', 
                           'state', 
                           'device', 
                           'date'])
#definindo plataforma, nesse caso apenas desktop
device='Desktop'
                                                ###############
                                                ###MAIN LOOP###
                                                ###############
for index, row in list_segments_df.iterrows():
    #segment_name = row['segment_name']
    segment_id = row['segment_id']
    company = row['company']
    vertical = row['vertical']
    state = row['state']
    url= 'https://api.similarweb.com/v1/segment/'+segment_id+'/traffic-and-engagement/query?api_key=c165c953a9fd44908addfcccec2c8caa&start_date='+start_date+'&end_date='+end_date+'&country=br&metrics=visits%2Cshare%2Cpages-per-visit%2Cpage-views%2Cbounce-rate%2Cvisit-duration%2Cunique-visitors&granularity=monthly&format=json&mtd=false'

    # Make the API request
    response = requests.get(url, headers={'api-key':'c165c953a9fd44908addfcccec2c8caa'})
    # Check if the request was successful
    if response.status_code == 200:
        json_data= response.json()
        data = json_data["segments"]
        df = pd.DataFrame(data)
        # Process the data as needed
        print('Data for '+company+'/'+vertical+'/'+state+' processed successfully!')
    else:
        print('API request failed for '+company+'/'+vertical+'/'+state+' :', response.status_code)

    for metric_name in s_metric_names:

        metric_data = df[metric_name]
        mdf = pd.DataFrame(metric_data)
        mdf["date"] = pd.to_datetime(df["date"])
        mdf=s_transform_dataframe(mdf, company, vertical, state, platform) 
        final_df=pd.concat([final_df, mdf], ignore_index=True) # apenda novos dados

    time.sleep(sleep_time_s)


#Ajustes de type
df=final_df
for col in df.columns:
    if df[col].dtype == 'object':
        df = df.astype({col:'string'})
for col in df.columns:
    if df[col].dtype == 'float64':
        df = df.astype({col:'float32'})
df.rename(columns={'date': 'dt'}, inplace=True)
df['dt'] = pd.to_datetime(df['dt'])
# Create 'year' and 'month' columns
df['year'] = df['dt'].dt.year.astype('int32')
df['month'] = df['dt'].dt.month.astype('int32')
# Remover o 'share' advindo da similarweb (vamos construir nosso próprio share )
df = df.loc[df['metric_name'] != 'share']
segment_states_df=df
print('segment analysis finished processing sucessfully!')
print(segment_states_df)

Data for CASAMINEIRA/REAL_ESTATE/16 processed successfully!
Data for CASAMINEIRA/REAL_ESTATE/BAHIA processed successfully!
Data for CASAMINEIRA/REAL_ESTATE/CEARÁ processed successfully!
Data for CASAMINEIRA/REAL_ESTATE/DISTRITO FEDERAL processed successfully!
Data for CASAMINEIRA/REAL_ESTATE/GOIÁS processed successfully!
Data for CASAMINEIRA/REAL_ESTATE/MINAS GERAIS processed successfully!
Data for CASAMINEIRA/REAL_ESTATE/PERNAMBUCO processed successfully!
Data for CASAMINEIRA/REAL_ESTATE/PARANÁ processed successfully!
Data for CASAMINEIRA/REAL_ESTATE/RIO DE JANEIRO processed successfully!
Data for CASAMINEIRA/REAL_ESTATE/RIO GRANDE DO SUL processed successfully!
Data for CASAMINEIRA/REAL_ESTATE/SANTA CATARINA processed successfully!
Data for CASAMINEIRA/REAL_ESTATE/SÃO PAULO processed successfully!
Data for CHAVESNAMAO/AUTO/16 processed successfully!
Data for CHAVESNAMAO/AUTO/BAHIA processed successfully!
Data for CHAVESNAMAO/AUTO/CEARÁ processed successfully!
Data for CHAVESNAMAO/AUT

In [4]:

#bloco para calculo de shares das empresas com multi verticais
df = segment_states_df

#pegar somente a métrica visits
df_visits = df.loc[df['metric_name'] == 'visits']
# Crie um novo DataFrame contendo apenas as linhas onde 'state' é 'FULL'
df_1 = df_visits.loc[df['state'] == 'FULL']
# Realize a soma agrupada por 'company', 'metric_name', 'dt', e 'vertical'
df_2 = df_1.groupby(['company', 'metric_name', 'dt', 'vertical'], as_index=False)['metric_value'].sum()
# Agrupe por 'company', 'dt' (agrupando a conta por verticais)  e calcule o percentual
df_2['share_per_vertical'] = df_2.groupby(['company', 'dt'])['metric_value'].transform(lambda x: (x / x.sum()) )
df_vertical=df_2
#df_verticals.to_csv('df_verticals.csv', index=False)




#bloco para calculo de shares das empresas com uma vertical apenas
df = segment_states_df
#pegar somente a métrica visits
df_v= df.loc[df['metric_name'] == 'visits']
# Retirar todas as linhas em que 'state' é igual a 'FULL'
df_2 = df_v.loc[df_v['state'] != 'FULL']
# Agrupe por 'company', 'dt' (agrupando a conta por verticais)  e calcule o percentual
df_2['share_per_state'] = df_2.groupby(['company', 'dt','vertical'])['metric_value'].transform(lambda x: (x / x.sum()) )
df_states=df_2


#bloco para calculo da tabela final de share por estados
#Consideraremos somente visits
#consideramos somente device=Total
df =df_website_analysis

#pegar somente a métrica visits
df_1 = df.loc[df['metric_name'] == 'visits']
#pegar somente device=totals
df_total = df_1.loc[df_1['device'] == 'Total']


######################################################## fazendo a divisão por verticais
# Filtrar linhas em df_total onde vertical é 'FULL' =empresas multivertical
df_full = df_total[df_total['vertical'] == 'FULL']
# Mesclar df_full com df_vertical usando as colunas 'company' e 'dt'
merged_df = pd.merge(df_full, df_vertical, on=['company', 'dt'])
# Calcular a nova coluna 'verticalized_visits'
merged_df['verticalized_visits'] = merged_df['metric_value_x'] * merged_df['share_per_vertical']
# Selecionar e renomear as colunas
df_result_selecionado = merged_df.loc[:, ['dt', 'metric_name_y', 'company', 'state', 'device', 'year', 'month', 'vertical_y', 'verticalized_visits']]
#dataframe com as empresas multi-vertical, já formatado para receber o próximo passo de dividir por estados
df_multis = df_result_selecionado.rename(columns={'vertical_y': 'vertical', 'verticalized_visits': 'metric_value','metric_name_y': 'metric_name',})


################################### Completando o dataframe para ser dividido por estados
# Filtrar linhas em df_total onde vertical é diferente de 'FULL' =empresas univertical
df_unis = df_total[df_total['vertical'] != 'FULL']
df_total_2=pd.concat([df_multis,df_unis], ignore_index=True) # tabela final sweb
#print(df_total_2)

######################################################## fazendo a divisão por estados
# Mesclar df_full com df_vertical usando as colunas 'company' e 'dt'
merged_df_2 = pd.merge(df_total_2, df_states, on=['company', 'dt','vertical'])
# Calcular a nova coluna 'stateded_visits'
merged_df_2['stated_visits'] = merged_df_2['metric_value_x'] * merged_df_2['share_per_state']

# Selecionar e renomear as colunas
df_result_selecionado = merged_df_2.loc[:, ['dt', 'metric_name_y', 'company', 'state_y', 'device_x', 'year_x', 'month_x', 'vertical', 'stated_visits']]
#dataframe com as empresas multi-vertical, já formatado para receber o próximo passo de dividir por estados
df_segment_analysis_visits = df_result_selecionado.rename(columns={'state_y': 'state', 'device_x': 'device','metric_name_y': 'metric_name', 'year_x': 'year', 'month_x': 'month', 'stated_visits': 'metric_value'})
print(df_segment_analysis_visits)




# Exibir o DataFrame resultante
#df_segment_analysis_visits.to_csv('teste_final.csv', index=False)





             dt metric_name     company              state device  year  \
0    2021-01-01      visits         OLX                 16  Total  2021   
1    2021-01-01      visits         OLX              BAHIA  Total  2021   
2    2021-01-01      visits         OLX              CEARÁ  Total  2021   
3    2021-01-01      visits         OLX   DISTRITO FEDERAL  Total  2021   
4    2021-01-01      visits         OLX              GOIÁS  Total  2021   
...         ...         ...         ...                ...    ...   ...   
3055 2023-10-01      visits  ZAPIMOVEIS             PARANÁ  Total  2023   
3056 2023-10-01      visits  ZAPIMOVEIS     RIO DE JANEIRO  Total  2023   
3057 2023-10-01      visits  ZAPIMOVEIS  RIO GRANDE DO SUL  Total  2023   
3058 2023-10-01      visits  ZAPIMOVEIS     SANTA CATARINA  Total  2023   
3059 2023-10-01      visits  ZAPIMOVEIS          SÃO PAULO  Total  2023   

      month     vertical  metric_value  
0         1         AUTO  9.191880e+06  
1         1      

/tmp/ipykernel_1442/3685944943.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2['share_per_state'] = df_2.groupby(['company', 'dt','vertical'])['metric_value'].transform(lambda x: (x / x.sum()) )


In [5]:
sweb_df=pd.concat([df_segment_analysis,df_website_analysis,df_segment_analysis_visits], ignore_index=True) # tabela final sweb

In [6]:
import io
import os

#import boto3
import s3fs

import pandas as pd
import pyarrow as pa

In [7]:
from dataeng_services.session import Session

session_provider = Session().client("session-provider")
aws_auth_dict = session_provider.get_temporary_credentials().json()

In [8]:
AWS_S3_FOLDER = 's3://olxbr-dl-cross-estrategia-e-planejamento/prod/cross_strategy/similarweb_traffic'
# csv_file_name = 'dataframe_s'

date_cols = ['dt']
int_cols = []
partition_col = ['dt']

In [9]:
#LENDO ARQUIVO ORIGINAL
df = sweb_df
# df = pd.read_csv(csv_file_name+'.csv')

df.head()

,dt,metric_name,metric_value,company,vertical,state,device,year,month
0,2021-01-01,visits,21745250.0,olx,Goods,All,Desktop,2021,1
1,2021-02-01,visits,18367332.0,olx,Goods,All,Desktop,2021,2
2,2021-03-01,visits,18811186.0,olx,Goods,All,Desktop,2021,3
3,2021-04-01,visits,17423282.0,olx,Goods,All,Desktop,2021,4
4,2021-05-01,visits,15321569.0,olx,Goods,All,Desktop,2021,5


In [10]:
# Coverte as colunas de data
for col in date_cols:
    df[ col ] = pd.to_datetime(df[ col ], format='%Y-%m-%d') ## Date   

# Coverte as colunas int
df[ int_cols ] = df[ int_cols ].round()

# Confirma os tipos corretos das Var na AWS
pa_schema = pa.Schema.from_pandas(df)
pa_schema = pa.schema([
pa.field(field.name, pa.date32()) if field.name in date_cols else field
for field in pa_schema
])

In [11]:
#Convertendo e salvando na aws
# save converted files localy
if len( partition_col ) > 0 :
    df.to_parquet(
            AWS_S3_FOLDER,
            schema=pa_schema,
            partition_cols=partition_col,
            storage_options={
                "key": aws_auth_dict['AccessKeyId'],
                "secret": aws_auth_dict['SecretAccessKey'],
                "token": aws_auth_dict['SessionToken'],
            },
                )
else:
    df.to_parquet(
            AWS_S3_FOLDER+csv_file_name+'.parquet', 
            schema=pa_schema, 
            storage_options={
                "key": aws_auth_dict['AccessKeyId'],
                "secret": aws_auth_dict['SecretAccessKey'],
                "token": aws_auth_dict['SessionToken'],
            },
                )

In [12]:
#adicionando partições à metastore
from dataeng_services.session import Session
from datetime import datetime,timedelta

# In order to avoid exposing credentials in your code
# You may use dataeng-cli --refresh-token first to create/refresh a token
# Or provide env variables OLXBR_INTERNAL_AUTH_USERNAME and OLXBR_INTERNAL_AUTH_PASSWORD

from dataeng_services.session import Session

metaservice = Session().client("metaservice")

In [13]:
# Dados do processo no ETLaaS
from dateutil.relativedelta import *
schema_name = 'cross_strategy'
table_name = 'similarweb_traffic'

# Periodo
start_date = '2021-01-01'
end_date = '2023-12-01'
date_step = relativedelta(months=+1) # mensal
# date_step = timedelta(days = 1)  # diario

In [14]:
date_list = []
day = start_date
while datetime.strptime( day, '%Y-%m-%d') <= datetime.strptime( end_date, '%Y-%m-%d'):
    date_list.append( day )
    day = str( datetime.strptime( day, '%Y-%m-%d') + date_step )[:10]
#     day = day[:8]+'01' # mensal
# dt=2023-02-01/
    
date_list

['2021-01-01',
 '2021-02-01',
 '2021-03-01',
 '2021-04-01',
 '2021-05-01',
 '2021-06-01',
 '2021-07-01',
 '2021-08-01',
 '2021-09-01',
 '2021-10-01',
 '2021-11-01',
 '2021-12-01',
 '2022-01-01',
 '2022-02-01',
 '2022-03-01',
 '2022-04-01',
 '2022-05-01',
 '2022-06-01',
 '2022-07-01',
 '2022-08-01',
 '2022-09-01',
 '2022-10-01',
 '2022-11-01',
 '2022-12-01',
 '2023-01-01',
 '2023-02-01',
 '2023-03-01',
 '2023-04-01',
 '2023-05-01',
 '2023-06-01',
 '2023-07-01',
 '2023-08-01',
 '2023-09-01',
 '2023-10-01',
 '2023-11-01',
 '2023-12-01']

In [15]:
# Adding partitions
response = metaservice.add_partition(
    schema_name=schema_name,
    table_name=table_name,
    partition_names=partition_col,
    partition_values=[ [i] for i in date_list],
)

# Response
response.content

b'{"detail":"The partitions have been added to the requested table!"}'